# Most Popular Pages in 2021

We gather the 50 most popular pages in 2021 for the annual end of year Medium post. The phab task for this is [T295943](https://phabricator.wikimedia.org/T295943)

## Configuration Variables

We gather page views for 2021 from the English Wikipedia, grabbing page titles from the most recent snapshot of `wmf_raw.mediawiki_page` at the time. I chose to gather the top 100 and will then limit the list further in a spreadsheet or otherwise, as the heuristics for excluding pages might develop along the way.

In [1]:
from wmfdata import spark

In [2]:
## The year we're gathering data for
year = 2021

## Name of the project in wmf.pageview_hourly
wiki = 'en.wikipedia'

## Database name and snapshot matching the project in wmf_raw.mediawiki_page
wiki_db = 'enwiki'
snapshot = '2021-11'

## The number of pages we return
top_n = 100

In [3]:
## Kudos to Connie for creating last year's query, which I used as a starting point!
## Added mobile view percentage as that helps determine whether specific articles 
## are disqualified list due to "automated" traffic not being caught by other heuristics.
## Added percentage of "none" referrers per T268262#6664186

pageview_query = f'''
WITH v AS (
    SELECT
        page_id,
        SUM(view_count) AS pageviews,
        SUM(IF(access_method = "mobile web", view_count, 0)) AS num_mobile_pageviews,
        100.0 * SUM(IF(access_method = "mobile web", view_count, 0)) / SUM(view_count)
            AS percent_mobile_pageviews,
        SUM(IF(referer_class = "none", view_count, 0)) AS num_noreferrer_pageviews,
        100.0 * SUM(IF(referer_class = "none", view_count, 0)) / SUM(view_count)
            AS percent_none_referrer
    FROM wmf.pageview_hourly
    WHERE year = {year}
        AND project = "{wiki}"
        AND namespace_id = 0 -- only article pages
        AND agent_type = "user" -- ignore bots, spiders, etc
    GROUP BY page_id
    ORDER by pageviews DESC
    LIMIT {top_n}
), p AS (
    SELECT
        page_id,
        page_title,
        page_latest
    FROM wmf_raw.mediawiki_page
    WHERE wiki_db = "{wiki_db}"
    AND snapshot = "{snapshot}"
    AND page_id IS NOT NULL
    AND page_namespace = 0
    AND NOT page_is_redirect
)
SELECT
    p.page_title,
    v.*
FROM v
LEFT JOIN p
ON v.page_id=p.page_id
'''

In [4]:
pageview_data = spark.run(pageview_query)

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


In [ ]:
pageview_data.sort_values('pageviews').tail(50)

In [6]:
pageview_data.to_csv('english-wikipedia-2021-top-pages-2021-12-17.csv')

Exception in thread Thread-5:
Traceback (most recent call last):
  File "/home/nettrom/.conda/envs/2021-05-03T16.30.23_nettrom/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/home/nettrom/.conda/envs/2021-05-03T16.30.23_nettrom/lib/python3.7/threading.py", line 1177, in run
    self.function(*self.args, **self.kwargs)
TypeError: stop_session() missing 1 required positional argument: 'session'

